# AI Agent Patterns with Groq: Reflection, Tool Use, Planning, and Collaboration

This Colab notebook explores key AI agent patterns – Reflection, Tool Use, Planning, and Multi-agent Collaboration – using the speed and capabilities of the Groq language model. We will define and experiment with Langchain agents powered by Groq to understand how these patterns enable intelligent and autonomous behavior.

**Objective:**

The primary goal of this notebook is to:

* Demonstrate the implementation of AI agents exhibiting Reflection, Tool Use, and Planning.
* Illustrate the potential for Multi-agent Collaboration through the interaction of specialized agents.
* Leverage the Groq LLM for fast and efficient processing within our agent workflows.

**Methodology:**

We will utilize the Langchain framework to define our agents and connect them to the Groq language model. The notebook will cover the following:

1.  **Setup:** Installation of necessary libraries (`langchain`, `langchain-groq`) and API key configuration for Groq.
2.  **Tool Definition:** Defining tools that our agents can use, such as a web search tool (DuckDuckGo).
3.  **Agent Definition:** Creating Langchain agents with specific roles and goals, focusing on the core AI agent patterns:
    * **Researcher Agent:** Employs Tool Use to gather information.
    * **Writer Agent:** Demonstrates Planning and generation of structured content based on research.
    * **Editor Agent:** Simulates Reflection by reviewing and providing feedback on the generated content.
4.  **Agent Execution:** Running the agents on specific tasks to observe their behavior and the application of the targeted patterns.
5.  **Exploration (Optional):** Further experimentation with different prompts, agent configurations, and the Groq model parameters.

**Key AI Agent Patterns Demonstrated:**

* **Tool Use:** The Researcher agent will utilize a search tool to access external information relevant to its tasks.
* **Planning:** The Writer agent will implicitly plan the structure and content of the blog post based on the provided research.
* **Reflection:** The Editor agent will perform a form of reflection by reviewing the generated text and identifying areas for improvement.
* **Multi-agent Collaboration:** While not a full `crew.ai` setup, the sequential interaction between the Researcher, Writer, and Editor demonstrates a basic form of collaborative workflow.

**Conclusion:**

This notebook provides a practical introduction to implementing key AI agent patterns using the Groq language model within the Langchain framework. By observing the behavior of these agents, we can gain a deeper understanding of how Reflection, Tool Use, Planning, and Collaboration contribute to the development of sophisticated AI systems.

**Note:** This notebook focuses on demonstrating the concepts using Langchain and Groq. For a dedicated `crew.ai` implementation, a separate setup and code structure would be required, which could be a next step in exploring multi-agent systems.

In [1]:
!pip install --upgrade --force-reinstall crewai crewai_tools groq langchain_community langchain-groq python-dotenv pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 530.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 14.5 

In [2]:
!pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.7 MB/s eta 0:00:00


In [3]:
import os
import getpass
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [4]:
try:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")
    print("Groq API Key set successfully.")
except Exception as e:
    print(f"Could not set Groq API key: {e}")

Enter your Groq API Key: ··········
Groq API Key set successfully.


In [5]:
# --- Tool Definition ---
# Using DuckDuckGo for searching - usually doesn't require an API key
search_tool = DuckDuckGoSearchRun()
print("Search tool initialized.")

# --- LLM Configuration ---
# Initialize the Groq LLM
# Common Models: "llama3-8b-8192", "llama3-70b-8192", "mixtral-8x7b-32768"
# Choose a model based on your needs (speed vs. capability)
# llama3-8b-8192 is very fast and capable for many tasks.
selected_model = "llama3-70b-8192"

try:
    llm = ChatGroq(
        api_key=os.environ.get("GROQ_API_KEY"),
        model=selected_model,
        temperature=0.7 # Adjust temperature for creativity vs. factuality
    )
    print(f"Groq LLM initialized with model: {selected_model}")
except Exception as e:
    print(f"Error initializing Groq LLM: {e}")
    print("Please ensure your API key is correct and valid.")
    llm = None # Set llm to None if initialization fails

Search tool initialized.
Groq LLM initialized with model: llama3-70b-8192


In [6]:
# --- Agent Definitions ---
if llm:
    from langchain.agents import ZeroShotAgent, AgentExecutor
    from langchain.prompts import PromptTemplate
    from langchain.chains import LLMChain

    # 1. Research Agent
    prefix = """Answer the following questions as best you can. You have access to the following tools:"""
    suffix = """Begin!

Question: {input}
{agent_scratchpad}"""
    prompt_researcher = ZeroShotAgent.create_prompt(
        tools=[search_tool],
        prefix=prefix,
        suffix=suffix,
        input_variables=["input", "agent_scratchpad"],
    )

    # Create the LLMChain
    llm_chain_researcher = LLMChain(llm=llm, prompt=prompt_researcher)

    # Initialize the ZeroShotAgent with the LLMChain
    researcher_agent = ZeroShotAgent(llm_chain=llm_chain_researcher, tools=[search_tool])
    researcher = AgentExecutor.from_agent_and_tools(agent=researcher_agent, tools=[search_tool], verbose=True)
    print("Researcher Agent defined using ZeroShotAgent with Groq.")

    # 2. Writing Agent (using LLMChain with Groq)
    writer_prompt = PromptTemplate.from_template("""You are an engaging tech content writer. Based on the following research:
{research}
Write a clear, engaging, and informative blog post about AI agent patterns.""")
    writer = LLMChain(llm=llm, prompt=writer_prompt, verbose=True)
    print("Writer Chain defined with Groq.")

    # 3. Editing Agent (using LLMChain with Groq)
    editor_prompt = PromptTemplate.from_template("""You are a meticulous tech editor. Review the following blog post for clarity, accuracy, grammar, style, and overall quality:
{draft}
Provide constructive feedback and ensure it meets publication standards.""")
    editor = LLMChain(llm=llm, prompt=editor_prompt, verbose=True)
    print("Editor Chain defined with Groq.")
else:
    print("Groq LLM not initialized. Cannot define agents.")

Researcher Agent defined using ZeroShotAgent with Groq.
Writer Chain defined with Groq.
Editor Chain defined with Groq.


<ipython-input-6-a272424c1d35>:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain_researcher = LLMChain(llm=llm, prompt=prompt_researcher)
<ipython-input-6-a272424c1d35>:24: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  researcher_agent = ZeroShotAgent(llm_chain=ll

In [7]:
if llm:
    researcher = AgentExecutor.from_agent_and_tools(
        agent=researcher_agent,
        tools=[search_tool],
        verbose=True,
        handle_parsing_errors=True  # Add this parameter
    )
    print(researcher.run("What are the key concepts and recent developments in Reflection, Tool Use, Planning, and Multi-agent Collaboration for AI agents?"))
else:
    print("Groq LLM is not initialized. Cannot run the researcher.")

<ipython-input-7-38802432154a>:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(researcher.run("What are the key concepts and recent developments in Reflection, Tool Use, Planning, and Multi-agent Collaboration for AI agents?"))




> Entering new AgentExecutor chain...
Here's my attempt to answer the question:

Question: What are the key concepts and recent developments in Reflection, Tool Use, Planning, and Multi-agent Collaboration for AI agents?

Thought: I need to start by searching for relevant information on AI agents and the specified concepts.

Action: duckduckgo_search
Action Input: "AI agents" "Reflection" "Tool Use" "Planning" "Multi-agent Collaboration"

Observation: In the dynamic and evolving field of artificial intelligence (AI), the development of intelligent agents capable of autonomous decision-making and problem-solving is a critical focus. Agentic reasoning patterns such as Reflection, Tool Use, Planning, and Multi-agent Collaboration form the foundation for creating sophisticated AI systems. Design Patterns for Building AI Agents: Reflection, Tool Use, Planning, and Multi-Agent Collaboration One of the key approaches to building effective AI systems is the use of design patterns. Here, we e

In [8]:
if llm:
    print(researcher.run("Multiple 3 with 15 and then add 35 to the result?"))
else:
    print("Groq LLM is not initialized. Cannot run the researcher.")



> Entering new AgentExecutor chain...
Let's solve this step by step.

Thought: I need to follow the order of operations (PEMDAS) to evaluate this expression.

Action: None (no search needed)
Action Input: None
Observation: None (no search needed) is not a valid tool, try one of [duckduckgo_search].
Thought:I apologize for the mistake. Since this question involves basic arithmetic operations, I don't need to use any search tool. I can simply evaluate the expression step by step.

Thought: I need to multiply 3 and 15 first.

Action: None (no search needed)
Action Input: None
Observation: None (no search needed) is not a valid tool, try one of [duckduckgo_search].
Thought:Thought: I can multiply 3 and 15 to get 45.

Action: None (no search needed)
Action Input: None
Observation: None (no search needed) is not a valid tool, try one of [duckduckgo_search].
Thought:Thought: I now need to add 35 to 45.

Action: None (no search needed)
Action Input: None
Observation: None (no search needed) 

In [9]:
if llm:
    print(researcher.run("what place is the considered as the technology hub of the world and why?"))
else:
    print("Groq LLM is not initialized. Cannot run the researcher.")



> Entering new AgentExecutor chain...
Here's my attempt to answer the question:

Question: what place is the considered as the technology hub of the world and why?
Thought: I need to find a reliable source that can provide information about the technology hub of the world.
Action: duckduckgo_search
Action Input: "what is the technology hub of the world"
Observation: London, United Kingdom is considered the leading tech hub in Europe and one of the world's most important technology and financial capitals. Is the UK a high tech country? Yes, the UK is regarded as a high-tech country, with significant contributions to innovation, technology, and the tech industry. What is the biggest tech hub in the world? The city's prestigious universities, such as the Southern University of Science and Technology, attract large numbers of IT students each year. 5. Tokyo, Japan. Tokyo is a global leader in robotics, electronics and advanced technology, with a strong focus on innovation. The city is fa